# 3.1 지도학습

### 3.1.1 K-최근접 이웃(K-nearest neighbor)
새로운 입력을 받았을 때, 기존 클러스터에서 모든 데이터와 인스턴스 기반 거리를 측정해서 가장 많은 속성을 갖는 클러스터에 할당하는 분류 알고리즘
- 기존 데이터들과 하나씩 거리를 계산하고 거리상 가장 가까운 데이터 k개를 선택하여 해당 클러스터에 할당
- k값에 따라 분류결과가 달라짐

In [3]:
# 라이브러리 호출 및 데이터 준비
import numpy as np                  # 벡터 및 행렬 연산 처리
import matplotlib.pyplot as plt     #  차트나 플롯을 그리는 라이브러리
import pandas as pd                 # 데이터 분석 및 조작을 위한 라이브러리
from sklearn import metrics         # 모델 성능 평가

names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

dataset = pd.read_csv('data/chap3/data/iris.data', names=names)

In [5]:
dataset

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [20]:
# 훈련과 검증 데이터셋 분리
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

In [21]:
# 모델 생성 및 훈련
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=50)

In [22]:
# 모델 정확도
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
print("정확도: {}".format(accuracy_score(y_test, y_pred)))

정확도: 0.8666666666666667


In [23]:
# 최적 K 찾기
k = 10
acc_array = np.zeros(k)
# k값 돌려가면서 정확도 저장
for k in np.arange(1, k+1, 1):
    classifier = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    acc_array[k-1] = acc
    
print(acc_array)
# 최대값 산출    
max_acc = np.amax(acc_array)
acc_list = list(acc_array)
k = acc_list.index(max_acc)
print("정확도", max_acc, "으로 최적의 k는", k+1, "입니다.")

[0.9 1.  1.  1.  1.  1.  1.  1.  1.  1. ]
정확도 1.0 으로 최적의 k는 2 입니다.


---
### 3.1.2 서포트 벡터 머신 (Supprot Vector Machine, SVM)
분류를 위한 기준선을 정의하는 모델로 새로운 데이터가 나타나면 결정 경계(기준선)를 기준으로 경계의 어느 쪽에 속하는지 분류하는 모델
- 결정 경계는 데이터가 분류된 클래스에서 최대한 멀리 떨어져 있을 때 가장 좋다.
- 마진(margin)
    - 결정경계와 서포트 벡터 사이의 거리
- 서포트 벡터(support vector)
    - 결정 경계와 가까이 있는 데이터
- 최적의 결정 경계는 마진을 최대로 해야한다.
- 최적 결정 경계를 위해 마진을 최대로 해야하는데, 이때 이상치(outlier)를 다루는 것이 중요
    - 하드 마진(hard margin) : 이상치를 허용하지 않은 마진
    - 소프트 마진 (sorft margin) : 이상치들이 마진 안에 포함되는 것을 허용한 마진


In [10]:
from sklearn import svm
from sklearn import metrics
from sklearn import datasets
from sklearn import model_selection
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

TF_CPP_MIN_LOG_LEVEL
- 환경변수. 로깅을 제어한다.
    - 0 : 기본값으로 모든 로그가 표시
    - 1 : INFO 로그 필터링
    - 2 : WARNING 로그를 필터링
    - 3 : ERROR 로그를 필터링

In [2]:
# iris 데이터를 준비하고 훈련과 검증 데이터셋으로 분리
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = model_selection.train_test_split(iris.data, iris.target, test_size=0.6, random_state=42)

In [12]:
# svm 모델에 대한 정확도
model = svm.SVC(kernel = 'linear', C=1.0, gamma=0.5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
score = metrics.accuracy_score(y_test, predictions)
print('정확도 : {0:f}'.format(score))

정확도 : 0.988889


- SVM은 선형 분류와 비선형 분류를 지원
- 보통 비선형 문제를 해결하는 기본적인 방법이 저차원 데이터를 고차원으로 보내는 것인데, 이는 많은 수학적 계산을 필요로 하므로 성능에 문제를 줄 수 있다.
- **커널트릭(kernel trick)**
    - 위의 문제를 해결하기 위한 방법
    - 선형 모델 -> 선형 커널
        $$K(a, b) = $$
    - 비선형 모델 -> 가우시안 RBF 커널, 다항식 커널(poly)
        - 벡터 내적을 계산한 후 고차원으로 보내는 방법으로 연산량을 줄임
        - 다항식 커널(polynomial kernel)
            - 실제로는 특성을 추가하지 않지만 엄청난 수의 특성 조합이 생기는 것과 같은 효과를 얻어 고차원으로 매핑이 가능.
        - 가우시안 RBF 커널(Gaussian 뀰 kernel)
            - 다항식 커널의 확장. 입력 벡터를 차원이 무한한 고차원으로 매핑.
            - 다항식 커널은 차수에 한계가 없지만, 가우시안 RBF는 차수에 제한 없이 무한한 확장 가능
- C
    - 오류를 어느 정도 허용할지 지정하는 파라미터.
    - 클수록 하드마진, 작을수록 소프트마진
- gamma
    - 결정 경계를 얼마나 유연하게 가져갈지 지정
    - gamma 값이 높으면 훈련 데이터에 많이 의존 -> 결정 경계가 곡선형태를 띠며 과적합

---
### 3.1.3 결정 트리 (Decision Tree)
- 왜 사용할까? : 데이터에 대한 **분류**
- 언제 사용하면 좋을까?
    - 이상치가 많은 값으로 구성된 데이터셋을 다룰 때
    - 머신 러닝이 어떤 방식으로 의사 결정을 하는지 알고 싶을 때
    
- 결정 트리는 데이터를 1차로 분류한 후, 각 영역의 순도(homogeneity)는 증가하고, 불순도(impurity)와 불확실성(uncertainty)은 감소하는 방향으로 학습을 진행한다.
- 순도가 증가하고 불확실성이 감소하는 것을 정보 획득(informaintion gain)이라고 한다.
- 순도 : 범주 안에서 같은 종류의 데이터만 모여 있는 상태
- 불순도 : 서로 다른 데이터가 섞여 있는 상태

- 순도 계산 방법
    - 엔트로피 (entropy)
        - 확률 변수의 불확실성을 수치로 나타낸 것
        - 높을수록 불확실성이 높다.
        - 엔트로피 = 0 -> 불확실성 최소 -> 순도 최대
        - 엔트로피 = 0.5 -> 불확실성 최대 -> 순도 최소
        - 레코드 m개가 A영역에 포함되어 있을 때 엔트로피 식
        $$ Entropy(A) = $$
    - 지니 계수(Gini index)
        - 불순도를 측정하는 지표
        - 데이터의 통계적 분산 정도를 정량화해서 나타낸 값.
        - 원소 n개 중에서 임의로 두 개를 추출했을 때, 추출된 두 개가 서로 다른 그룹에 속해 있을 확률을 의미
        - 지니 계수가 높을수록 데이터가 분산 되어있다.
        $$ G(S) = $$
        
        - 로그를 계산할 필요가 없어 엔트로피보다 계산이 빠르다. 이 때문에 결정트리에서 엔트로피보다 많이 사용
        

In [17]:
# 라이브러리 호출 및 데이터 준비
import pandas as pd
df = pd.read_csv('data/chap3/titanic/train.csv', index_col='PassengerId')
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [18]:
# 데이터 전처리
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
df['Sex'] = df['Sex'].map({'male':0, 'femail':1})    # 남자는 0, 여자는 1로 변환
df = df.dropna()
X = df.drop('Survived', axis=1)
y = df['Survived']

C:\Users\NICE\AppData\Local\Temp\ipykernel_63624\2816975619.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'] = df['Sex'].map({'male':0, 'femail':1})    # 남자는 0, 여자는 1로 변환


In [19]:
# 훈련과 검증 데이터셋으로 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [21]:
# 결정 트리 모델 생성
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [22]:
# 모델 훈련
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [23]:
# 모델 예측
y_predict = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.7982456140350878

In [24]:
# 혼동행렬을 이용한 성능 측정
from sklearn.metrics import confusion_matrix
pd.DataFrame(
                confusion_matrix(y_test, y_predict),
                columns = ['Predicted Not Survival', 'Predicted Survival'],
                index=['True Not Survival', 'True Survival']
)

,Predicted Not Survival,Predicted Survival
True Not Survival,82,13
True Survival,10,9
